In [13]:
import requests
import re
import nltk
import random
import pandas as pd

from nltk import FreqDist
from nltk import word_tokenize

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams

from bs4 import BeautifulSoup
from urllib import request

from url_config import *
from utilitools import *

In [29]:

def tokenizar_url(url):
    #Se envia solicitud a la página
    resultado=requests.get(url)

    #Se solicita el texo
    content=resultado.text
    soup=BeautifulSoup(content, 'lxml')

    if 'espectador' in url.lower(): box = bs_espectador(soup)
    elif 'semana' in url.lower(): box = bs_semana(soup)
    elif 'larepublica' in url.lower(): box = bs_larepublica(soup)
    elif 'portafolio' in url.lower(): box = bs_portafolio(soup)

    else: return 'no config'

    tokenizer=RegexpTokenizer('\w+')
    tokens=tokenizer.tokenize(box)
    tokens=[token.lower() for token in tokens]

    return tokens

def top_freqdst(tokens,n=100):
    stops = stopwords.words('spanish')

    filtered_tokens = [token for token in tokens if token not in stops]

    f_dst = FreqDist(filtered_tokens)
    top_n = f_dst.most_common(n)

    return (top_n)

def riq_lex(tokens):
    vocabulario = sorted(set(tokens))
    return len(vocabulario)/len(tokens)

def top_ngrams_freqdist(tokens,n=2,m=100):
    n_grams = ngrams(tokens, n)
    
    token_grams = [ ' '.join(elms) for elms in n_grams]
    
    f_dst = FreqDist(token_grams)
    top_m = f_dst.most_common(m) 
    return (top_m)


In [129]:
def atributos(tokens):
    atrib = {}

    top_freqdist = top_freqdst(tokens,n=100)
    for x in top_freqdist:
        atrib['top_freqdist({})'.format(x[0])] = x[1]
    
    n_gram_freqdist = top_ngrams_freqdist(tokens,n=2,m=100)
    for x in n_gram_freqdist:
        atrib['top_ngram({})'.format(x[0])] = x[1]

    atrib['riq_lex'] = riq_lex(tokens)

    return atrib


In [21]:
data = leer_paths('data')
recomendador = leer_paths('recomendador')
scripts = leer_paths('scripts')

df = pd.read_excel(os.path.join(data,'url_noticias_clasificadas.xlsx'))
df = df[['url','categoria']]
df['tokens'] = df['url'].apply(tokenizar_url)

In [148]:
fset = [(atributos(texto), clase) for texto, clase in zip(df['tokens'].values, df['categoria'].values)]
random.shuffle(fset)
train, test = fset[:10], fset[10:]
classifier = nltk.NaiveBayesClassifier.train(train)
print(nltk.classify.accuracy(classifier, test)) #se mide el accuracy

0.625


In [36]:
url='https://www.elespectador.com/turismo/en-colombia-el-turismo-sostenible-paso-de-ser-una-tendencia-a-una-exigencia/'
tokens = tokenizar_url(url)
a = top_ngrams_freqdist(tokens,n=5)
for x in a:
    print(x[0],x[1])

el ministerio de comercio industria 4
ministerio de comercio industria y 4
de comercio industria y turismo 4
el primer foro de turismo 2
primer foro de turismo sostenible 2
en el jardín botánico de 2
comercio industria y turismo procolombia 2
industria y turismo procolombia y 2
liderada por el ministerio de 2
por el ministerio de comercio 2
foro de turismo sostenible se 1
de turismo sostenible se realizará 1
turismo sostenible se realizará en 1
sostenible se realizará en el 1
se realizará en el jardín 1
realizará en el jardín botánico 1
el jardín botánico de bogota 1
jardín botánico de bogota foto 1
botánico de bogota foto jardín 1
de bogota foto jardín botánico 1
bogota foto jardín botánico la 1
foto jardín botánico la sostenibilidad 1
jardín botánico la sostenibilidad es 1
botánico la sostenibilidad es uno 1
la sostenibilidad es uno de 1
sostenibilidad es uno de los 1
es uno de los componentes 1
uno de los componentes principales 1
de los componentes principales de 1
los componentes 

In [149]:
def leer_csv(path,csv_file):
    '''
    Inserta ruta de carpeta donde se encuentra el archivo y nombre de archivo para hacer lectura de archivo como DataFrame
    '''
    for file in os.listdir(path):
        file_path = os.path.join(path,file)

        if csv_file in file_path:
            df = pd.read_csv(file_path)
    return df

df_noticias = leer_csv(data,'noticias.csv')


In [150]:
df_noticias['tokens'] = df_noticias['news_text_content'].apply(word_tokenize)

,news_id,news_url_absolute,news_init_date,news_final_date,news_title,news_text_content,tokens
0,news10006,https://www.bluradio.com/economia/precio-dolar...,2022-07-30,2022-08-14,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ...","[Este, martes, ,, 2, de, agosto, ,, el, dolar,..."
1,news10011,https://www.semana.com/economia/macroeconomia/...,2022-07-30,2022-08-14,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...,"[No, hay, nada, mas, permanente, que, lo, temp..."
2,news10015,https://elcomercio.pe/respuestas/que/gustavo-p...,2022-07-30,2022-08-14,Gustavo Petro: ?Que dice el informe final de s...,El equipo de empalme del gobierno del presiden...,"[El, equipo, de, empalme, del, gobierno, del, ..."
3,news10028,https://www.lanacion.com.ar/estados-unidos/cua...,2022-07-30,2022-08-14,Cuanto cuesta y como enviar dinero de Estados ...,?Buscas opciones para el envio de dinero de Es...,"[?, Buscas, opciones, para, el, envio, de, din..."
4,news10029,https://www.lanacion.com.ar/estados-unidos/com...,2022-07-30,2022-08-14,Como enviar dinero de Estados Unidos a otros p...,?Buscas opciones para el envio de dinero de Es...,"[?, Buscas, opciones, para, el, envio, de, din..."
...,...,...,...,...,...,...,...
23372,news99983,https://www.tuuputchika.com/2022/08/07/como-av...,2022-07-30,2022-08-14,?Como avanza La Guajira en materia de producci...,"Por: Mesa Mas La Guajira. En La Guajira, el 54...","[Por, :, Mesa, Mas, La, Guajira, ., En, La, Gu..."
23373,news99989,https://www.publimetro.com.mx/nacional/2022/08...,2022-07-30,2022-08-14,Mineros: ?En que condiciones laboran los traba...,A pesar de que se enfrentan a multiples riesgo...,"[A, pesar, de, que, se, enfrentan, a, multiple..."
23374,news99991,https://www.laopinion.com.co/columnistas/la-de...,2022-07-15,2022-07-30,"La deuda externa, un problema mayor","Durante los anos 60 y 70 del siglo pasado, soc...","[Durante, los, anos, 60, y, 70, del, siglo, pa..."
23375,news99996,https://www.laopinion.com.co/premium/economia/...,2022-07-15,2022-07-30,?Por que ocurren las recesiones economicas?,El comportamiento de la economia en los ultimo...,"[El, comportamiento, de, la, economia, en, los..."


In [151]:
df_noticias['clase'] = df_noticias['tokens'].apply(lambda x: classifier.classify(atributos(x)))

,news_id,news_url_absolute,news_init_date,news_final_date,news_title,news_text_content,tokens,clase
0,news10006,https://www.bluradio.com/economia/precio-dolar...,2022-07-30,2022-08-14,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ...","[Este, martes, ,, 2, de, agosto, ,, el, dolar,...",Regulaciones
1,news10011,https://www.semana.com/economia/macroeconomia/...,2022-07-30,2022-08-14,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...,"[No, hay, nada, mas, permanente, que, lo, temp...",Innovación
2,news10015,https://elcomercio.pe/respuestas/que/gustavo-p...,2022-07-30,2022-08-14,Gustavo Petro: ?Que dice el informe final de s...,El equipo de empalme del gobierno del presiden...,"[El, equipo, de, empalme, del, gobierno, del, ...",Regulaciones
3,news10028,https://www.lanacion.com.ar/estados-unidos/cua...,2022-07-30,2022-08-14,Cuanto cuesta y como enviar dinero de Estados ...,?Buscas opciones para el envio de dinero de Es...,"[?, Buscas, opciones, para, el, envio, de, din...",Sostenibilidad
4,news10029,https://www.lanacion.com.ar/estados-unidos/com...,2022-07-30,2022-08-14,Como enviar dinero de Estados Unidos a otros p...,?Buscas opciones para el envio de dinero de Es...,"[?, Buscas, opciones, para, el, envio, de, din...",Sostenibilidad
...,...,...,...,...,...,...,...,...
23372,news99983,https://www.tuuputchika.com/2022/08/07/como-av...,2022-07-30,2022-08-14,?Como avanza La Guajira en materia de producci...,"Por: Mesa Mas La Guajira. En La Guajira, el 54...","[Por, :, Mesa, Mas, La, Guajira, ., En, La, Gu...",Innovación
23373,news99989,https://www.publimetro.com.mx/nacional/2022/08...,2022-07-30,2022-08-14,Mineros: ?En que condiciones laboran los traba...,A pesar de que se enfrentan a multiples riesgo...,"[A, pesar, de, que, se, enfrentan, a, multiple...",Innovación
23374,news99991,https://www.laopinion.com.co/columnistas/la-de...,2022-07-15,2022-07-30,"La deuda externa, un problema mayor","Durante los anos 60 y 70 del siglo pasado, soc...","[Durante, los, anos, 60, y, 70, del, siglo, pa...",Sostenibilidad
23375,news99996,https://www.laopinion.com.co/premium/economia/...,2022-07-15,2022-07-30,?Por que ocurren las recesiones economicas?,El comportamiento de la economia en los ultimo...,"[El, comportamiento, de, la, economia, en, los...",Innovación


In [153]:
df_noticias.drop(['tokens'], axis=1)
df_noticias.to_excel(os.path.join(data,'output\\categorizacion.xlsx'),index=False)